# Algorithme de région de confiance basique

In [118]:
using BenchmarkTools
using LinearAlgebra

verbose = false

false

Nous allons regrouper les constantes dans la structure `BasicTrustRegion`.

In [96]:
struct BasicTrustRegion{T <: Real}
    η1:: T
    η2:: T
    γ1:: T
    γ2:: T
end

function BTRDefaults()
    return BasicTrustRegion(0.01,0.9,0.5,0.5)
end

# We define a type to store the state at a given iteration.
mutable struct BTRState
    iter::Int
    x::Vector
    xcand::Vector
    g::Vector
    step::Vector
    Δ::Float64
    ρ::Float64

    tol::Float64

    trace
    keepTrace::Bool
    
    function BTRState()
        state = new()
        state.tol = 1e-6
        state.keepTrace = false
        return state
    end
end

In [97]:
function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    # If the iteration is successful, update the iterate
    if (state.ρ >= b.η1)
        return true
    else
        return false
    end
end

acceptCandidate! (generic function with 1 method)

In [98]:
function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if (state.ρ >= b.η2)
        stepnorm = norm(state.step)
        state.Δ = min(1e20,max(4*stepnorm,state.Δ))
    elseif (state.ρ >= b.η1)
        state.Δ *= b.γ2
    else
        state.Δ *= b.γ1
    end
end

updateRadius! (generic function with 1 method)

## Algorithme de région de confiance de base

In [99]:
function btr(f::Function, g!::Function, H!::Function, Step::Function,
    x0::Vector, state:: BTRState = BTRState(), ApproxH::Bool = false, verbose::Bool = false)
    
    b = BTRDefaults()
    state.iter = 0
    state.x = x0
    n=length(x0)

    tol2 = state.tol*state.tol
    
    state.g = zeros(n)
    # A better initialization procedure should be used with quasi-Newton approximations
    # We could rely on some preconditioner.
    H = zeros(n,n)+I
    
    fx = f(x0)
    g!(x0, state.g)
    state.Δ = 0.1*norm(state.g) # 1.0

    if (ApproxH)
        y = zeros(n)
        gcand = zeros(n)
        # H!(H, y, state.step)
    else
        H!(x0, H)
    end
    
    nmax = 1000
    if (state.keepTrace)
        state.trace= x0'
    end
    
    function model(s::Vector, g::Vector, H::Matrix)
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    while (dot(state.g,state.g) > tol2 && state.iter < nmax)
        # Compute the step by approximately minimize the model
        state.step = Step(state.g, H, state.Δ)
        state.xcand = state.x+state.step
        
        # Compute the actual reduction over the predicted reduction
        fcand = f(state.xcand)
        state.ρ = (fcand-fx)/(model(state.step, state.g, H))

       if (ApproxH)
            g!(state.xcand, gcand)
            y = gcand-state.g
            sy = dot(state.step,y)
#            if (sy < 1e-6)
#                println(state.iter, ". ", state.ρ, " ", state.Δ, " ", norm(state.step), " ", (model(state.step, state.g, H)), " ", norm(y), " ", sy, " ", norm(state.g))
#            end
            H = H!(H, y, state.step)
        end

        if (acceptCandidate!(state, b))
            state.x = copy(state.xcand)
            if (ApproxH == false)
                g!(state.x, state.g)
                H!(state.x, H)
            else
                state.g = copy(gcand)
            end
            fx = fcand
        end

        if (state.keepTrace)
            state.trace= [state.trace ; state.x']
        end
        
        updateRadius!(state, b)
        state.iter += 1
    end
    
    return state
end

btr (generic function with 4 methods)

In [100]:
function CauchyStep(g::Vector, H::Matrix, Δ::Float64)
    q = dot(g,H*g)
    normg = norm(g)
    if (q <= 0)
        τ = 1.0
    else
        τ = min((normg*normg*normg)/(q*Δ),1.0)
    end
    return -τ*g*Δ/normg
end

CauchyStep (generic function with 1 method)

In [102]:
function BFGSUpdate(B::Matrix, y::Vector, s::Vector)
    sy = dot(s,y)
    if (sy > 1e-10)
        Bs = B*s
        B = B - (Bs*Bs')/dot(s, Bs) + (y*y')/sy
    end
    return B
end

BFGSUpdate (generic function with 1 method)

## Gradient conjugué tronqué

In [103]:
function stopCG(normg::Float64, normg0::Float64, k::Int, kmax::Int, χ::Float64 = 0.1, θ::Float64 = 0.5)
    if ((k == kmax) || (normg <= normg0*min(χ, normg0^θ)))
        if (verbose)
            println("CG stops after $k iterations")
        end
        return true
    else
        return false
    end
end

stopCG (generic function with 3 methods)

In [104]:
function TruncatedCG(g::AbstractVector{T}, H::AbstractMatrix{T}, Δ::Float64) where T
    n = length(g)
    s = zeros(n)

    normg0 = norm(g)
    v = g
    d = -v
    gv = dot(g,v)
    norm2d = gv
    norm2s = 0
    sMd = 0
    k = 0
    Δ2 = Δ*Δ

    while (stopCG(norm(g), normg0, k, n) == false)
        Hd = H*d
        κ = dot(d,Hd)
 
        # Is the curvature negative in the direction d?
        if (κ <= 0)
            if (k == 0)
                s = d/norm(d)*Δ
            else
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s,s))))/norm2d
            s += σ*d
            end
            break
        end

        α = gv/κ

        # Check is the model minimizer is outside the trust region
        norm2s += α*(2*sMd+α*norm2d)
        if (norm2s >= Δ2)
            if (k == 0)
                s = d/norm(d)*Δ
            else
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s,s))))/norm2d
            s += σ*d
            end
            break
        end

        # The model minimizer is inside the trust region
        s += α*d
        g += α*Hd
        v = g
        newgv = dot(g,v)
        β = newgv/gv
        gv = newgv
        d = -v+β*d
        
        sMd = β*(sMd+α*norm2d)
        norm2d = gv+β*β*norm2d
        
        k += 1;
    end
    
    return s
end

TruncatedCG (generic function with 2 methods)

## Exemple

In [105]:
function rosenbrock(x::Vector)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

function rosenbrock_gradient!(x::Vector, storage::Vector)
    storage[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    storage[2] = 200.0 * (x[2] - x[1]^2)
end

function rosenbrock_hessian!(x::Vector, storage::Matrix)
    storage[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    storage[1, 2] = -400.0 * x[1]
    storage[2, 1] = -400.0 * x[1]
    storage[2, 2] = 200.0
end

rosenbrock_hessian! (generic function with 1 method)

In [106]:
defaultState = BTRState()

BTRState(381668912, #undef, #undef, #undef, #undef, 4.375495003e-315, 1.885686596e-315, 1.0e-6, #undef, false)

In [107]:
defaultState.tol

1.0e-6

In [108]:
state = btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, CauchyStep, [0,0])

BTRState(778, [0.9999990671653469, 0.9999981306218827], [0.9999990671653469, 0.9999981306218827], [-3.817981544068388e-7, -7.419362679783603e-7], [-5.970170605448271e-8, 3.0722752931776427e-8], 0.5578315950889445, 1.0000000474084514, 1.0e-6, #undef, false)

In [109]:
state = btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, TruncatedCG, [0,0])

CG stops after 2 iterations
CG stops after 2 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 1 iterations
CG stops after 2 iterations


BTRState(20, [0.9999999999717916, 0.9999999999083069], [0.9999999999717916, 0.9999999999083069], [1.4054073682203108e-8, -7.055245276887945e-9], [5.939376790494293e-6, 1.1902429996984944e-5], 0.31807381938434265, 1.0000047382691175, 1.0e-6, #undef, false)

In [110]:
defaultState.tol = 1e-4

0.0001

In [111]:
state = btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, CauchyStep, [0,0], defaultState)

BTRState(772, [0.9999574528409513, 0.9999147382962227], [0.9999574528409513, 0.9999147382962227], [-1.7418821383525582e-5, -3.3839188118278685e-5], [-2.648300225813218e-6, 1.364048575608009e-6], 0.5578315950889445, 1.0000021026865589, 0.0001, #undef, false)

In [112]:
state = btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, TruncatedCG, [0,0], defaultState)

CG stops after 2 iterations
CG stops after 2 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 1 iterations

BTRState(19, [0.9999940605950011, 0.99998809747831], [0.9999940605950011, 0.99998809747831], [-2.3800789251098873e-6, -4.7493937449516466e-6], [-1.8479138998099901e-6, 9.280457471001435e-7], 0.31807381938434265, 1.0000014746706374, 0.0001, #undef, false)


CG stops after 2 iterations
CG stops after 1 iterations


In [113]:
@benchmark btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, CauchyStep, [0,0], defaultState)

BenchmarkTools.Trial: 
  memory estimate:  772.36 KiB
  allocs estimate:  19301
  --------------
  minimum time:     2.264 ms (0.00% GC)
  median time:      2.981 ms (0.00% GC)
  mean time:        3.732 ms (1.70% GC)
  maximum time:     20.060 ms (0.00% GC)
  --------------
  samples:          1339
  evals/sample:     1

In [119]:
@benchmark btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, TruncatedCG, [0,0], defaultState)

BenchmarkTools.Trial: 
  memory estimate:  33.98 KiB
  allocs estimate:  602
  --------------
  minimum time:     60.000 μs (0.00% GC)
  median time:      81.300 μs (0.00% GC)
  mean time:        98.582 μs (2.49% GC)
  maximum time:     4.344 ms (97.49% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [120]:
state = btr(rosenbrock, rosenbrock_gradient!, BFGSUpdate, TruncatedCG, [0,0], defaultState, true)

BTRState(24, [0.9999994477848443, 0.9999986767618864], [0.9999994477848443, 0.9999986767618864], [8.641876423233189e-5, -4.376162143771012e-5], [0.0001536090193170501, 0.00030842583564268263], 0.2061077379303815, 1.0045406013896907, 0.0001, #undef, false)

In [121]:
@benchmark btr(rosenbrock, rosenbrock_gradient!, BFGSUpdate, TruncatedCG, [0,0], defaultState, true)

BenchmarkTools.Trial: 
  memory estimate:  70.94 KiB
  allocs estimate:  1121
  --------------
  minimum time:     118.100 μs (0.00% GC)
  median time:      155.899 μs (0.00% GC)
  mean time:        183.391 μs (3.04% GC)
  maximum time:     5.278 ms (96.35% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [122]:
state = btr(rosenbrock, rosenbrock_gradient!, BFGSUpdate, CauchyStep, [0,0], defaultState, true)

BTRState(1000, [0.9985494826368874, 0.9970979511351126], [0.9985494826368874, 0.9970979511351126], [-0.0016555881791385726, -0.0006236278565774356], [-1.2524281286954046e-6, 3.3245906367962425e-6], 0.11966392415732387, 0.9999705932797386, 0.0001, #undef, false)

"Trust-Region Methods", Introduction
$$
f(x,y) = -10x^2+10y^2+4\sin(xy)-2x+x^4
$$

In [123]:
using ForwardDiff
defaultState.tol = 1e-6

1.0e-6

In [124]:
f(x::Vector) = -10*x[1]^2+10*x[2]^2+4*sin(x[1]*x[2])-2*x[1]+x[1]^4

f (generic function with 1 method)

In [125]:
g = x -> ForwardDiff.gradient(f, x);
H = x -> ForwardDiff.hessian(f, x)

function g!(x::Vector, storage::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [126]:
function H!(x::Vector, storage::Matrix)
    s = H(x)
    n, m = size(s)
    storage[1:n,1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [127]:
state = btr(f, g!, H!, CauchyStep, [0,0])

BTRState(10, [2.306630127652979, -0.3323086483268177], [2.306630127652979, -0.3323086483268177], [-1.9235244508308824e-9, 1.4039548190680762e-8], [-2.6188011584984253e-8, -3.5879612974955863e-9], 0.9876566190586658, 1.3810396868170254, 1.0e-6, #undef, false)

In [128]:
state = btr(f, g!, H!, TruncatedCG, [0,0])

BTRState(9, [2.306630127704536, -0.3323086487344874], [2.306630127704536, -0.3323086487344874], [4.522604513113038e-11, -9.29194499121877e-11], [2.6931294018039604e-7, -2.159080370597743e-6], 0.9876566190586676, 1.0000768267343858, 1.0e-6, #undef, false)

In [129]:
state = btr(f, g!, BFGSUpdate, CauchyStep, [0,0], BTRState(), true)

BTRState(15, [2.3066301409821888, -0.3323086308230537], [2.3066301409821888, -0.3323086308230537], [5.99808672063773e-7, 6.325624042347044e-7], [1.2207055456836203e-6, 3.2358698696715524e-8], 0.13856970761632884, 0.9885297883047421, 1.0e-6, #undef, false)

In [130]:
state = btr(f, g!, BFGSUpdate, TruncatedCG, [0,0], BTRState(), true)

BTRState(16, [2.3066301277215193, -0.33230864871689997], [2.3066301277215193, -0.33230864871689997], [8.083631541921932e-10, 5.312683626357284e-10], [3.069381018283615e-8, -9.583908845702118e-7], 0.4040086781927015, 1.000152209177207, 1.0e-6, #undef, false)

In [131]:
f([2.30663, -0.332309])

-31.18073338518543

In [132]:
state = btr(f, g!, H!, TruncatedCG, [-2.0,-2.0], BTRState())

BTRState(9, [-2.2102195200834442, 0.32974845699582234], [-2.2102195200834442, 0.32974845699582234], [-2.2014745582055184e-10, 6.418865439172805e-12], [2.881632201245532e-6, 8.295428793163734e-8], 0.577574285844766, 0.9999275749910249, 1.0e-6, #undef, false)

In [133]:
f([-2.21022, 0.329748])

-22.14296062774464

In [134]:
st = BTRState()
st.keepTrace = true

true

In [135]:
state = btr(f, g!, BFGSUpdate, TruncatedCG, [-2.0,-2.0], st, true)

BTRState(13, [-2.210219508107465, 0.3297484485240419], [-2.210219508107465, 0.3297484485240419], [4.5694093131487534e-7, -2.671945287247013e-7], [6.73008627569298e-8, -6.780444184062432e-7], 0.061287803810465384, 0.9864583922033531, 1.0e-6, [-2.0 -2.0; -2.0 -2.0; … ; -2.2102195754083276 0.3297491265684603; -2.210219508107465 0.3297484485240419], true)

In [136]:
st.trace

14×2 Array{Float64,2}:
 -2.0      -2.0
 -2.0      -2.0
 -2.1974   -0.183741
 -2.1974   -0.183741
 -2.1974   -0.183741
 -2.43193   0.597025
 -2.08508   0.244409
 -2.19216   0.304055
 -2.20917   0.340282
 -2.21238   0.328408
 -2.20988   0.329765
 -2.21022   0.329742
 -2.21022   0.329749
 -2.21022   0.329748

## Rosenbrock généralisé

Les exemples précédents travaillent sur des fonctions à deux dimensions, limitant la capacité d'évaluer les gains du gradient conjugué tronqué. Reprenons la fonction de Rosenbrock généralisée
$$
f(x) = \sum_{i = 1}^{n-1} \left( 100(x_{i+1}^2-x_i)^2 + (x_i-1)^2 \right).
$$
avec l'implémentation du gradient et la matrice Hessienne, avec un stockage dense ou creux.

In [137]:
function rosenbrock(x::Vector)
    return sum(100*(x[i+1]^2 - x[i])^2 + (x[i] - 1)^2 for i in 1:length(x)-1)
end

function ∇f(x:: Vector)
    n = length(x)
    g = zeros(n)
    for i = 1:n-1
        g[i] = -200*(x[i+1]^2-x[i])+2*(x[i]-1)
    end
    for i = 2:n
        g[i] += 400*x[i]*(x[i]^2-x[i-1])
    end
    return g
end

function Hess(x:: Vector)
    n = length(x)
    H = zeros(n,n)
    H[1,1] = 202
    for i = 2:n
        H[i,i-1] = H[i-1,i] = -400*x[i]
        H[i,i] = 400*(x[i]^2-x[i-1])+800*x[i]^2 + 202
    end
    H[n,n] -= 202
    return H
end

function TriHess(x)
    n = length(x)
    d = zeros(n)
    d[1] = 202
    d[2:n] = [400*(x[i]^2-x[i-1])+800*x[i]^2 + 202 for i = 2:n]
    d[n] -= 202
    dl = [-400*x[i] for i = 2:n]
    H = SymTridiagonal(d, dl)
    return H
end

TriHess (generic function with 1 method)

In [161]:
function ∇f!(x:: Vector, g:: Vector)
    g[:] = ∇f(x)
end

function Hess!(x:: Vector, H:: Matrix)
    H[:,:] = Hess(x)
end

function TriHess!(x:: Vector, H:: AbstractMatrix{T}) where T
    n = length(x)
    d = zeros(n)
    d[1] = 202
    d[2:n] = [400*(x[i]^2-x[i-1])+800*x[i]^2 + 202 for i = 2:n]
    d[n] -= 202
    dl = [-400*x[i] for i = 2:n]
    H = SymTridiagonal(d, dl)
end

TriHess! (generic function with 2 methods)

In [163]:
n = 1000

x = 10*ones(n)
g = zeros(n)
H = zeros(n,n)

∇f!(x, g)
Hess!(x, H)

H = []
TriHess!(x, H)
H

LoadError: DimensionMismatch("tried to assign 1000000 elements to 0 destinations")

In [140]:
n = 1000
verbose = true

state = btr(rosenbrock, ∇f!, Hess!, TruncatedCG, x, BTRState())

CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 2 iterations
CG stops after 4 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 3 iterations
CG stops after 5 iterations
CG stops after 13 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 4 iterations


BTRState(43, [0.9999999999667143, 0.9999999999976941, 0.9999999999936121, 0.9999999999731043, 0.9999999999766485, 0.9999999999892332, 1.0000000000079827, 1.0000000000112708, 0.9999999999848341, 0.9999999999571358  …  0.9999999999980563, 0.9999999999981336, 0.9999999999982345, 0.9999999999983632, 0.9999999999985217, 0.9999999999987004, 0.9999999999988892, 0.9999999999990907, 0.9999999999993094, 0.9999999999995385], [0.9999999999667143, 0.9999999999976941, 0.9999999999936121, 0.9999999999731043, 0.9999999999766485, 0.9999999999892332, 1.0000000000079827, 1.0000000000112708, 0.9999999999848341, 0.9999999999571358  …  0.9999999999980563, 0.9999999999981336, 0.9999999999982345, 0.9999999999983632, 0.9999999999985217, 0.9999999999987004, 0.9999999999988892, 0.9999999999990907, 0.9999999999993094, 0.9999999999995385], [-5.801339630906455e-9, 1.3558894007532826e-8, 5.279990444888689e-9, -1.505372604298022e-8, -8.333230105776306e-9, -4.640838602306743e-9, 7.797118950577535e-9, 1.416656081338798

In [141]:
state.x

1000-element Array{Float64,1}:
 0.9999999999667143
 0.9999999999976941
 0.9999999999936121
 0.9999999999731043
 0.9999999999766485
 0.9999999999892332
 1.0000000000079827
 1.0000000000112708
 0.9999999999848341
 0.9999999999571358
 0.9999999999310264
 0.9999999999605184
 0.9999999999668127
 ⋮
 0.9999999999980019
 0.9999999999980098
 0.9999999999980563
 0.9999999999981336
 0.9999999999982345
 0.9999999999983632
 0.9999999999985217
 0.9999999999987004
 0.9999999999988892
 0.9999999999990907
 0.9999999999993094
 0.9999999999995385

In [142]:
state.g

1000-element Array{Float64,1}:
 -5.801339630906455e-9
  1.3558894007532826e-8
  5.279990444888689e-9
 -1.505372604298022e-8
 -8.333230105776306e-9
 -4.640838602306743e-9
  7.797118950577535e-9
  1.4166560813387984e-8
 -2.5588424709928945e-9
 -9.294095714052233e-9
 -3.6173233072963776e-8
  1.303775088048576e-9
 -9.64677782064709e-9
  ⋮
 -4.1764303126805803e-10
 -4.1747183487770136e-10
 -4.049220958520466e-10
 -3.864710773613355e-10
 -3.677285143043788e-10
 -3.4257996439536263e-10
 -3.0664693006976177e-10
 -2.666102894005319e-10
 -2.2943980049164987e-10
 -1.905564595003552e-10
 -1.4362289135747842e-10
 -9.299228054256019e-11

In [88]:
@benchmark btr(rosenbrock, ∇f!, Hess!, TruncatedCG, x, BTRState())

CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 2 iterations
CG stops after 4 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 3 iterations
CG stops after 5 iterations
CG stops after 13 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 4 iterations


CG stops after 5 iterations
CG stops after 13 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 4 iterations
CG stops after 12 iterations
CG stops after 6 iterations
CG stops after 16 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 2 iterations
CG stops after 4 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations


CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 3 iterations
CG stops after 5 iterations
CG stops after 13 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 4 iterations
CG stops after 12 iterations
CG stops after 6 iterations
CG stops after 16 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 2 iterations
CG stops after 4 iterations
CG stops after 11 iterations
C

CG stops after 16 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 2 iterations
CG stops after 4 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 3 iterations
CG stops after 5 iterations
CG stops after 13 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations

CG stops after 11 iterations
CG stops after 3 iterations
CG stops after 5 iterations
CG stops after 13 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 12 iterations
CG stops after 3 iterations
CG stops after 4 iterations
CG stops after 12 iterations
CG stops after 6 iterations
CG stops after 16 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 2 iterations
CG stops after 4 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
CG stops after 11 iterations
C

BenchmarkTools.Trial: 
  memory estimate:  235.64 MiB
  allocs estimate:  4424
  --------------
  minimum time:     314.916 ms (7.38% GC)
  median time:      344.322 ms (6.95% GC)
  mean time:        355.682 ms (9.66% GC)
  maximum time:     453.234 ms (29.42% GC)
  --------------
  samples:          15
  evals/sample:     1

CG stops after 16 iterations


In [143]:
x

1000-element Array{Float64,1}:
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
  ⋮
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0

In [144]:
state

BTRState(43, [0.9999999999667143, 0.9999999999976941, 0.9999999999936121, 0.9999999999731043, 0.9999999999766485, 0.9999999999892332, 1.0000000000079827, 1.0000000000112708, 0.9999999999848341, 0.9999999999571358  …  0.9999999999980563, 0.9999999999981336, 0.9999999999982345, 0.9999999999983632, 0.9999999999985217, 0.9999999999987004, 0.9999999999988892, 0.9999999999990907, 0.9999999999993094, 0.9999999999995385], [0.9999999999667143, 0.9999999999976941, 0.9999999999936121, 0.9999999999731043, 0.9999999999766485, 0.9999999999892332, 1.0000000000079827, 1.0000000000112708, 0.9999999999848341, 0.9999999999571358  …  0.9999999999980563, 0.9999999999981336, 0.9999999999982345, 0.9999999999983632, 0.9999999999985217, 0.9999999999987004, 0.9999999999988892, 0.9999999999990907, 0.9999999999993094, 0.9999999999995385], [-5.801339630906455e-9, 1.3558894007532826e-8, 5.279990444888689e-9, -1.505372604298022e-8, -8.333230105776306e-9, -4.640838602306743e-9, 7.797118950577535e-9, 1.416656081338798

In [150]:
x = 10*ones(1000)

1000-element Array{Float64,1}:
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
  ⋮
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0

In [151]:
state = btr(rosenbrock, ∇f!, BFGSUpdate, TruncatedCG, x, st, true)

CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 2 iterations
CG stops after 4 iterations
CG stops after 3 iterations
CG stops after 3 iterations
CG stops after 5 iterations
CG stops after 4 iterations
CG stops after 6 iterations
CG stops after 6 iterations
CG stops after 8 iterations
CG stops after 10 iterations
CG stops after 5 iterations
CG stops after 13 iterations
CG stops after 8 iterations
CG stops after 3 iterations
CG stops after 7 iterations
CG stops after 2 iterations
CG stops after 3 iterations
CG stops after 6 iterations
CG stops after 2 iterations
CG stops after 4 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 i

CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 ite

CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 ite

CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 iterations
CG stops after 1 ite

BTRState(1000, [11.461717008511425, 3.4001979194124337, 1.844424040266735, 1.3576915751820129, 1.1611294029255248, 1.0770449381249998, 1.0345782912296002, 1.0151745830338958, 1.0065570174566378, 1.0034825297282053  …  0.999964508662121, 0.9999005181541686, 0.9998364206727796, 0.9997970398178428, 0.9997825084864721, 0.999769323976993, 0.9997613070662522, 0.9997998168024567, 0.9998905953183191, 0.9999712656453181], [11.461717008511425, 3.4001979194124337, 1.844424040266735, 1.3576915751820129, 1.1611294029255248, 1.0770449381249998, 1.0345782912296002, 1.0151745830338958, 1.0065570174566378, 1.0034825297282053  …  0.999964508662121, 0.9999005181541686, 0.9998364206727796, 0.9997970398178428, 0.9997825084864721, 0.999769323976993, 0.9997613070662522, 0.9997998168024567, 0.9998905953183191, 0.9999712656453181], [0.9976574839791894, 139.9631394792027, 3.1641465538542497, 2.0133045759365498, -3.855417941327742, 1.0171768441477629, -1.9007197723024274, -1.1899553782134278, -0.8832330785622307

In [147]:
for i = 1:100
    println(state.trace[i*1001])
end

1.0000779076406534
1.0000389302579686
1.0000194799687934
1.000009734893736
1.0000048691901269
1.000002435156645
1.000001212459858
1.0000006179617515
1.0000003009999585
1.0000001562622258
1.000000068571528
1.0000000368718827
1.0000000233765927
1.0000000066355041
1.0000000083652223
1.0000000030569676
1.0000000013130697
0.999999997298334
0.9999999964743498
1.0000000141235899
0.9999999888866509
0.9999999919664408
1.000000020693744
0.9999999827262495
1.0000000038966754
1.0000000120739672
0.9999999848555502
1.0000000018201192
1.0000000183333613
0.999999977983532
1.0000000037432233
1.000000012586133
0.9999999800575977
1.0000000140241683
1.0000000075745639
0.9999999775832217
1.0000000184721975
0.9999999944755716
0.9999999958308902
1.0000000027941771
1.0000000044114472
0.9999999910382867
1.000000007625155
0.9999999971904188
1.0000000039564279
0.9999999903679624
1.0000000060468883
1.0000000089302479
0.9999999763891954
1.000000024141731
0.9999999929008454
0.9999999870003424
1.0000000204309063
0.9

In [148]:
state.x

1000-element Array{Float64,1}:
 1.0000779076406534
 1.0000389302579686
 1.0000194799687934
 1.000009734893736
 1.0000048691901269
 1.000002435156645
 1.000001212459858
 1.0000006179617515
 1.0000003009999585
 1.0000001562622258
 1.000000068571528
 1.0000000368718827
 1.0000000233765927
 ⋮
 1.0000000234349506
 0.9999999868167105
 1.0000000002216969
 1.0000000035104808
 0.9999999925832239
 1.0000000030793124
 1.000000003224948
 1.0000000011757433
 0.9999999903342167
 1.000000007017036
 1.0000000029106562
 0.9999999956398746

In [149]:
state.g

1000-element Array{Float64,1}:
  0.00016493711153131585
  5.3604327814005505e-5
  5.30011173241475e-5
  1.4733081580925974e-5
  1.09166591926481e-5
  7.369043346881389e-6
 -6.362074117753407e-6
  1.3813888865820311e-5
 -8.087603087900555e-6
  8.746165224974762e-6
 -8.544971251854238e-6
  1.663786747381017e-7
  6.0203913952154295e-6
  ⋮
  3.400928778667869e-5
 -2.2672314699494114e-5
  4.0912637441086505e-6
  6.395533445723968e-6
 -1.0067526842162978e-5
  4.76220220042687e-6
  1.5293757304094714e-6
  3.7544288238655593e-6
 -1.2962226465278451e-5
  9.733121051699187e-6
  1.8537131948837827e-6
 -4.652362741418559e-6

In [ ]:
@benchmark btr(rosenbrock, ∇f!, Hess!, TruncatedCG, x, BTRState())

state = btr(rosenbrock, rosenbrock_gradient!, BFGSUpdate, TruncatedCG, [0,0], defaultState, true)

In [152]:
mutable struct NotYetComputedMatrix{T} <: AbstractMatrix{T}
    x::Vector{T}
    f::Function
    function NotYetComputedMatrix(f::Function, T::Type = Float64)
        n = new{T}()
        n.f = f
        return n
    end
end

function H!(x::Vector{T}, stack::NotYetComputedMatrix{T}) where T
    stack.x = x
    stack
end
import Base.*
function *(h::NotYetComputedMatrix{T}, v::AbstractVector{T}) where T
    return h.f(h.x, v)
end

import Base.show
show(io::IO, mine::MIME{Symbol("text/plain")}, x::NotYetComputedMatrix{T}) where T = show(io, "NYCM{$T}")

show (generic function with 262 methods)